In [ ]:
import rdflib
import time
import pandas as pd
import os
import s3fs
from unidecode import unidecode
from dotenv import load_dotenv
from ydata_profiling import ProfileReport
from utils.dcat_reader_ckan import DCATReaderCKAN
from utils.utils import create_universe_pprn

load_dotenv()  # take environment variables from .env.

In [ ]:
import glob
import pandas as pd
import json

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

# Enter S3 cmd copied form SSLab : 

#######

data = []
iteration = 0
BUCKET = "projet-ecolab-action-qualite"
FILE_KEY_S3 = "DUMP_INTEGRATION_JSON/integration_page_107_2023-10-14.json"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3


print(len(fs.ls(BUCKET + '/DUMP_INTEGRATION_JSON')))
      
for graph in fs.ls(BUCKET + '/DUMP_INTEGRATION_JSON'):
    if '.json' in graph:
        with fs.open(graph, mode="rb") as file_in:
            read_data = file_in.read()
            page_data = DCATReaderCKAN(read_data).get_data()
            # print("Fichier: ", graph, "- Nombre de datasets : ", len(page_data))
            data.append(
                page_data
            )
            iteration += 1

# print("Nombre de pages traitées : ", iteration)
data = pd.concat(data)


In [ ]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql+psycopg2://{os.environ['POSTGRES_USER']}:{os.environ['POSTGRES_PASSWORD']}@{os.environ['POSTGRES_URL']}:{os.environ['POSTGRES_PORT']}/{os.environ['POSTGRES_DB']}")

with engine.begin() as connection:
    data.to_sql(name='datasets', con=connection, if_exists='replace', index=False)

In [ ]:
import sweetviz as sv

my_report = sv.analyze(data)
my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

## Attribution des univers

In [ ]:
def create_universe_pprn(row):
    """
    For a specific row, return if the element is considered to be a part of the 'PPRN' universe
    """
    for elem in ['title', 'description']:
        if 'pprn' in row[elem].lower():
            return 'PPRN'
        elif 'prevention des risques' in unidecode(row[elem]).lower():
            return 'PPRN'
    return None

In [ ]:
data['univers'] = data.apply(create_universe_pprn, axis=1)

In [ ]:
data['univers'].count()

In [ ]:
2357 PPRN avec filtre title = PPRN
3985 PPRN en ajoutant le filtre title = 'prevention des risques' (independament des accents et majuscules)
5122 PPRN en appliquant également ces filtre au champ 'description'